In [109]:
import arcpy, os
from arcpy import env
import geopandas as gpd
import pandas as pd
import numpy as np

In [32]:
env.overwriteOutput = True

In [3]:
path = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\PerformanceAnalysis'

In [4]:
arcpy.env.workspace = os.path.join(path, "SystemCompleteness/SystemCompleteness.gdb")

In [5]:
network = os.path.join(path, "network.shp")

In [6]:
facilities = os.path.join(path, 'sidewalk_bikeway_trails/facilities.shp')

In [65]:
network_gpd = gpd.read_file(network)

In [83]:
network_gpd.head()

,id,name,type,fed_class,length,Length_mi,geometry
0,210,AYRES,RD,Major Collector,0.0,0.105734,"LINESTRING (-13701910.120 5481217.713, -137019..."
1,215,AYRES,RD,Major Collector,0.0,0.080414,"LINESTRING (-13701615.348 5481206.862, -137016..."
2,239,RIVER,AVE,Major Collector,0.0,0.191666,"LINESTRING (-13705016.630 5480409.094, -137049..."
3,254,RIVER,AVE,Major Collector,0.0,0.145884,"LINESTRING (-13704575.406 5480372.173, -137045..."
4,259,TERRY,ST,Major Collector,0.0,0.050614,"LINESTRING (-13713458.691 5479775.646, -137134..."


In [72]:
network_gpd['Length_mi'].sum()

258.7563811014173

In [67]:
facilities_gpd = gpd.read_file(facilities)

In [68]:
facilities_gpd.head()

,id,mode,geometry
0,1,bikeway,"LINESTRING (-13705805.396 5477828.642, -137058..."
1,2,bikeway,"LINESTRING (-13710446.573 5474013.658, -137104..."
2,3,bikeway,"LINESTRING (-13701510.497 5474278.039, -137015..."
3,4,bikeway,"LINESTRING (-13701521.145 5469198.971, -137014..."
4,5,bikeway,"LINESTRING (-13705017.230 5480591.833, -137050..."


In [100]:
list(facilities_gpd['mode'].unique())

['bikeway', 'sidewalk', 'trail']

In [7]:
arcpy.MakeFeatureLayer_management(facilities,"bikeway", "mode = 'bikeway'")

<Result 'bikeway'>

In [82]:
arcpy.MakeFeatureLayer_management(network, "network")

<Result 'network'>

In [9]:
fieldList = arcpy.ListFields("network")
field_names = [f.name for f in fieldList]
newfield = "Length_mi"
if newfield in field_names:
    pass
else:
    arcpy.AddField_management("network", newfield, "DOUBLE", "", "", 100)

In [10]:
arcpy.CalculateGeometryAttributes_management("network", [["Length_mi", "LENGTH_GEODESIC"]], "MILES_US")

<Result 'network'>

In [37]:
arcpy.SelectLayerByLocation_management("network", "INTERSECT", "bikeway", "70 Feet")

id,value
0,a Layer object
1,network
2,2541


In [38]:
arcpy.Statistics_analysis("network", "network_w_bikeway", [["Length_mi", "SUM"]])

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\PerformanceAnalysis\\SystemCompleteness/SystemCompleteness.gdb\\network_w_bikeway'>

In [42]:
arcpy.SelectLayerByAttribute_management("network", "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


In [19]:
fields = [f.name for f in arcpy.ListFields("network_w_bikeway")]

In [20]:
fields

['OBJECTID', 'FREQUENCY', 'SUM_Length_mi']

In [39]:
arr = arcpy.da.TableToNumPyArray("network_w_bikeway", fields)

In [40]:
table = pd.DataFrame(arr, columns=fields)

In [62]:
round(table['SUM_Length_mi'].values[0], 2)

227.04

In [187]:
def network_with_facility(facility_mode, level):
    arcpy.MakeFeatureLayer_management(facilities, facility_mode, "mode = '{0}'".format(facility_mode))
    arcpy.MakeFeatureLayer_management(network, "network")
    arcpy.CalculateGeometryAttributes_management("network", [["Length_mi", "LENGTH_GEODESIC"]], "MILES_US")
    
    if level == 'regionwide':
        arcpy.SelectLayerByLocation_management("network", "INTERSECT", facility_mode, "70 Feet")

    elif level == 'equity-focused areas':
        equity_area = os.path.join(path, 'service_transit_equity/equity_area.shp')
        arcpy.MakeFeatureLayer_management(equity_area, "equity_area")
        arcpy.SelectLayerByLocation_management("network", "COMPLETELY_WITHIN", "equity_area")
        arcpy.SelectLayerByLocation_management("network", "INTERSECT", facility_mode, "70 Feet", "SUBSET_SELECTION")
    else:
        if level == '1/4 miles from transit stops':
            transit_stops = os.path.join(path, 'service_transit_equity/transit_stops.shp')
            arcpy.MakeFeatureLayer_management(transit_stops, "transit_stops")
        else:
            high_freq_transit = os.path.join(path, 'service_transit_equity/high_frequency_transit.shp')
            arcpy.MakeFeatureLayer_management(high_freq_transit, "transit_stops")
        
        arcpy.SelectLayerByLocation_management("network", "WITHIN_A_DISTANCE_GEODESIC", "transit_stops", "0.25 Miles")
        arcpy.SelectLayerByLocation_management("network", "INTERSECT", facility_mode, "70 Feet", "SUBSET_SELECTION")       
        
    
    arcpy.Statistics_analysis("network", "network_w_" + facility_mode, [["Length_mi", "SUM"]])
    fields = [f.name for f in arcpy.ListFields("network_w_" + facility_mode)]
    arr = arcpy.da.TableToNumPyArray("network_w_" + facility_mode, fields)
    table = pd.DataFrame(arr, columns=fields)
    return round(table['SUM_Length_mi'].values[0], 2)  

In [91]:
network_with_facility('bikeway', level = 'regionwide')

227.04

In [75]:
network_with_facility('sidewalk', level = 'regionwide')

202.05

In [76]:
network_with_facility('trail', level = 'regionwide')

12.27

In [188]:
network_with_facility('trail', level = '1/4 miles from transit stops')

10.0

In [93]:
network_with_facility('sidewalk', level = '1/4 miles from transit stops')

171.26

In [92]:
network_with_facility('bikeway', level = '1/4 miles from transit stops')

179.41

In [95]:
network_with_facility('bikeway', level = 'equity-focused areas')

42.29

In [96]:
network_with_facility('sidewalk', level = 'equity-focused areas')

43.99

In [97]:
network_with_facility('trail', level = 'equity-focused areas')

3.17

In [172]:
network_with_facility('bikeway', level = '1/4 miles from high frequency transit stops')

94.27

In [173]:
network_with_facility('trail', level = '1/4 miles from high frequency transit stops')

7.83

In [174]:
network_with_facility('sidewalk', level = '1/4 miles from high frequency transit stops')

100.83

In [190]:
levels = ['regionwide', '1/4 miles from transit stops', 
          '1/4 miles from high frequency transit stops', 'equity-focused areas']

In [101]:
modes = list(facilities_gpd['mode'].unique())

In [184]:
def get_completeness(facility_mode, level):
    a = network_with_facility(facility_mode, level)
    return a, round((a/258.76 * 100), 2)

In [185]:
get_completeness('bikeway', level = 'regionwide')

(227.04, 87.74)

In [191]:
for level in levels:
    for mode in modes:
        print("Mode: " + mode + "; Level: " + level)
        print(get_completeness(mode, level))

Mode: bikeway; Level: regionwide
(227.04, 87.74)
Mode: sidewalk; Level: regionwide
(202.05, 78.08)
Mode: trail; Level: regionwide
(12.27, 4.74)
Mode: bikeway; Level: 1/4 miles from transit stops
(179.41, 69.33)
Mode: sidewalk; Level: 1/4 miles from transit stops
(171.26, 66.18)
Mode: trail; Level: 1/4 miles from transit stops
(10.0, 3.86)
Mode: bikeway; Level: 1/4 miles from high frequency transit stops
(94.27, 36.43)
Mode: sidewalk; Level: 1/4 miles from high frequency transit stops
(100.83, 38.97)
Mode: trail; Level: 1/4 miles from high frequency transit stops
(7.83, 3.03)
Mode: bikeway; Level: equity-focused areas
(42.29, 16.34)
Mode: sidewalk; Level: equity-focused areas
(43.99, 17.0)
Mode: trail; Level: equity-focused areas
(3.17, 1.23)


In [103]:
transit_stops_freq = os.path.join(path, 'service_transit_equity/stops_frequency.shp')

In [105]:
transit_stops_freq_gpd = gpd.read_file(transit_stops_freq)

In [106]:
transit_stops_freq_gpd.head()

,number,name,longitude,latitude,ons,geometry
0,00001,E/S of 58th N of Main,-122.926727,44.046333,15416.0,POINT (-13684142.077 5472616.286)
1,00002,E/S of 58th S of D,-122.926224,44.049454,16027.0,POINT (-13684085.793 5473099.581)
2,00004,E/S of 58th S of Thurston Rd,-122.926247,44.052246,6427.0,POINT (-13684088.398 5473531.989)
3,00006,S/S of Thurston W of 64th,-122.916626,44.052948,10110.0,POINT (-13683018.050 5473640.676)
4,00007,S/S of Thurston E of 65th,-122.913231,44.052673,1498.0,POINT (-13682639.441 5473598.174)


In [107]:
transit_stops_freq_gpd['ons'].describe()

count      1096.000000
mean      12199.741788
std       45142.536625
min           0.000000
25%         846.250000
50%        2766.500000
75%        7546.250000
max      881282.000000
Name: ons, dtype: float64

In [160]:
gdf = transit_stops_freq_gpd.dropna()

In [146]:
gdf = gdf[gdf['ons'] >= np.percentile(gdf['ons'].values, 75)]

In [147]:
gdf.shape[0]

274

In [150]:
transit_stops_freq_gpd['ons'].mean()

12199.741788321167

In [161]:
gdf['ons'].mean()

12199.741788321167

In [162]:
gdf = gdf[gdf['ons'] >= gdf['ons'].mean()]

In [166]:
gdf.shape[0]/transit_stops_freq_gpd.shape[0]

0.16047297297297297

In [169]:
gdf.to_file(os.path.join(path, "service_transit_equity/high_frequency_transit.shp"))